# BigFrames Performance Analytical Report

## Why BigFrames is slow (b/440407542)

- Add `@perf_inspect.runtime_logger` to the call methods, such as

In [1]:
import bigframes.pandas as bpd
import bigframes.bigquery as bbq
import time

In [2]:
start_time = time.monotonic()
search_query = bpd.DataFrame({"query_id": ["dog", "cat"],
                              "embedding": [[1.0, 2.0], [3.0, 5.2]]})
print("Runtime: {:.2f} seconds".format(time.monotonic() - start_time))

|--DataFrame.__init__ started at 2118257.94 seconds
|----get_global_session started at 2118257.94 seconds
|------Session.__init__ started at 2118257.94 seconds
|--------ClientsProvider.__init__ started at 2118257.94 seconds
|----------_get_default_credentials_with_project started at 2118257.94 seconds


/usr/local/google/home/chelsealin/src/bigframes1/bigframes/session/__init__.py:2287: DefaultLocationWarning: No explicit location is set, so using location US for the session.
  return Session(context)


|----------_get_default_credentials_with_project ended at 2118259.89 seconds. Runtime: 1.95 seconds
|--------ClientsProvider.__init__ ended at 2118259.94 seconds. Runtime: 2.00 seconds
|------Session.__init__ ended at 2118260.40 seconds. Runtime: 2.47 seconds
|----get_global_session ended at 2118260.40 seconds. Runtime: 2.47 seconds
|----DataFrame.__init__ started at 2118260.42 seconds
|----DataFrame.__init__ ended at 2118260.42 seconds. Runtime: 0.00 seconds
|--DataFrame.__init__ ended at 2118260.42 seconds. Runtime: 2.49 seconds
Runtime: 2.49 seconds


In [3]:
start_time = time.monotonic()
df = bbq.vector_search(
    base_table="bigframes-dev.bigframes_tests_sys.base_table",
    column_to_search="my_embedding",
    query=search_query,
    top_k=2
).sort_values("id")
print("Runtime: {:.2f} seconds".format(time.monotonic() - start_time))

|--vector_search started at 2118260.43 seconds
|----DataFrame.__init__ started at 2118260.44 seconds
|----DataFrame.__init__ ended at 2118260.44 seconds. Runtime: 0.00 seconds
|----DataFrame.__init__ started at 2118261.18 seconds
|----DataFrame.__init__ ended at 2118261.18 seconds. Runtime: 0.00 seconds
|----DataFrame.__init__ started at 2118261.18 seconds
|----DataFrame.__init__ ended at 2118261.18 seconds. Runtime: 0.00 seconds
|--vector_search ended at 2118261.18 seconds. Runtime: 0.74 seconds
|--DataFrame.__init__ started at 2118261.18 seconds
|--DataFrame.__init__ ended at 2118261.18 seconds. Runtime: 0.00 seconds
Runtime: 0.74 seconds


In [4]:
start_time = time.monotonic()
print(df.to_pandas())
print("Runtime: {:.2f} seconds".format(time.monotonic() - start_time))

|--DataFrame.to_pandas started at 2118261.19 seconds
|----BigQueryCachingExecutor.execute started at 2118261.19 seconds
|----BigQueryCachingExecutor.execute ended at 2118261.45 seconds. Runtime: 0.26 seconds
|--DataFrame.to_pandas ended at 2118261.46 seconds. Runtime: 0.27 seconds
  query_id  embedding  id my_embedding  distance
0      dog    [1. 2.]   1      [1. 2.]       0.0
1      cat  [3.  5.2]   2      [2. 4.]   1.56205
0      dog    [1. 2.]   4    [1.  3.2]       1.2
1      cat  [3.  5.2]   5    [5.  5.4]  2.009975
Runtime: 0.29 seconds


In [5]:
start_time = time.monotonic()
print(df)
print("Runtime: {:.2f} seconds".format(time.monotonic() - start_time))

|--DataFrame.__repr__ started at 2118261.49 seconds
|----Block.retrieve_repr_request_results started at 2118261.49 seconds
|------BigQueryCachingExecutor.execute started at 2118261.49 seconds


|------BigQueryCachingExecutor.execute ended at 2118267.25 seconds. Runtime: 5.75 seconds
|------BigQueryCachingExecutor.execute started at 2118267.25 seconds
|------BigQueryCachingExecutor.execute ended at 2118267.64 seconds. Runtime: 0.39 seconds
Time taken to execute head: 0.39 seconds
|------BigQueryCachingExecutor.execute started at 2118267.64 seconds
|------BigQueryCachingExecutor.execute ended at 2118267.64 seconds. Runtime: 0.00 seconds
Time taken to execute row_count: 0.00 seconds
Time taken to execute to_pandas: 0.00 seconds
|----Block.retrieve_repr_request_results ended at 2118267.65 seconds. Runtime: 6.15 seconds
|--DataFrame.__repr__ ended at 2118267.65 seconds. Runtime: 6.16 seconds
  query_id  embedding  id my_embedding  distance
0      dog    [1. 2.]   1      [1. 2.]       0.0
1      cat  [3.  5.2]   2      [2. 4.]   1.56205
0      dog    [1. 2.]   4    [1.  3.2]       1.2
1      cat  [3.  5.2]   5    [5.  5.4]  2.009975

[4 rows x 5 columns]
Runtime: 6.16 seconds
